# Lego 3D

This Jupyter notebook demonstrates making a simple 3d game with Jupylet.

Run the notebook and click the game canvas once it shows up to bring it into focus and then use the arrows and keys W, A, S, D, SHIFT, ALT and CAPSLOCK to control the camera or the moon.

* Moon textures are by [NASA](https://svs.gsfc.nasa.gov/4720).  
* Nebula sky was generated with [spacescape](https://github.com/petrocket/spacescape) by Alex Peterson.  
* Alien texture is from [TextureHaven](https://texturehaven.com/textures/).
* Eye texture is from [Flickr](https://www.flickr.com/photos/filterforge/33680912465/in/photostream/) stated as CC BY 2.0.

In [1]:
import logging
import random
import struct
import time
import glm
import sys
import os

In [2]:
sys.path.insert(0, os.path.abspath('./..'))

In [3]:
from jupylet.label import Label
from jupylet.app import App
from jupylet.state import State
from jupylet.loader import load_blender_gltf

In [4]:
logger = logging.getLogger()

In [5]:
app = App(768, 512)

In [6]:
scene = load_blender_gltf('./scenes/lego/lego.gltf')

In [7]:
scene.shadows = True

In [8]:
camera = scene.cameras['Camera']

In [9]:
brick = scene.meshes['brick.green']

In [10]:
state = State(
    
    capslock = False,
    shift = False,
    alt = False,
    
    up = False,
    down = False,
    right = False,
    left = False,
    
    key_w = False,
    key_s = False,
    key_a = False,
    key_d = False,
    
    lv = glm.vec3(0),
    av = glm.vec3(0),
)

In [11]:
@app.event
def key_event(key, action, modifiers):
    logger.info('Enter key_event(key=%r, action=%r, modifiers=%r).', key, action, modifiers)
    
    keys = app.window.keys

    value = action == keys.ACTION_PRESS
    
    if key == keys.CAPS_LOCK and value:
        state.capslock = not state.capslock
        
    state.alt = modifiers.alt        
    state.shift = modifiers.shift
        
    if key == keys.SPACE:
        state.lv *= 0.
        state.av *= 0.
        
    if key == keys.UP:
        state.up = value
        
    if key == keys.DOWN:
        state.down = value
        
    if key == keys.LEFT:
        state.left = value
        
    if key == keys.RIGHT:
        state.right = value
        
    if key == keys.W:
        state.key_w = value
        
    if key == keys.S:
        state.key_s = value    
        
    if key == keys.A:
        state.key_a = value
        
    if key == keys.D:
        state.key_d = value


In [12]:
obj = brick if state.capslock else camera

In [13]:
linear_acceleration = 1 / 2
angular_acceleration = 1 / 24

In [14]:
@app.run_me_every(1/48)
def move_object(ct, dt):
        
    global obj
    
    obj = brick if state.capslock else camera
    sign = -1 if obj is camera else 1
    
    if state.right and state.shift:
        state.av.z += angular_acceleration * sign
            
    if state.right and not state.shift:
        state.av.y -= angular_acceleration
        
    if state.left and state.shift:
        state.av.z -= angular_acceleration * sign
            
    if state.left and not state.shift:
        state.av.y += angular_acceleration
            
    if state.up:
        state.av.x -= angular_acceleration
        
    if state.down:
        state.av.x += angular_acceleration
        
    if state.key_w and state.alt:
        state.lv.y += linear_acceleration
                
    if state.key_w and not state.alt:
        state.lv.z += linear_acceleration * sign
                
    if state.key_s and state.alt:
        state.lv.y -= linear_acceleration
        
    if state.key_s and not state.alt:
        state.lv.z -= linear_acceleration * sign
        
    if state.key_a:
        state.lv.x += linear_acceleration * sign
        
    if state.key_d:
        state.lv.x -= linear_acceleration * sign
        
    state.lv = glm.clamp(state.lv, -64, 64)
    state.av = glm.clamp(state.av, -64, 64)
    
    obj.move_local(dt * state.lv)
    
    obj.rotate_local(dt * state.av.x, (1, 0, 0))
    obj.rotate_local(dt * state.av.y, (0, 1, 0))
    obj.rotate_local(dt * state.av.z, (0, 0, 1))
    
    state.lv *= 0.67 ** dt
    state.av *= 0.67 ** dt

In [15]:
label0 = Label('Hello World!', color='white', font_size=12, x=10, y=74)
label1 = Label('Hello World!', color='white', font_size=12, x=10, y=52)
label2 = Label('Hello World!', color='white', font_size=12, x=10, y=30)
label3 = Label('Hello World!', color='white', font_size=12, x=10, y=8)

hello_world = Label('hello, world 3D!', color='cyan', font_size=24, x=575, y=10)

In [16]:
@app.event
def render(ct, dt):
        
    app.window.clear()
        
    scene.draw()
    
    label0.text = 'time to draw - %.2f ms' % (1000 * app._time2draw_rm)
    label1.text = 'up - %r' % obj.up
    label2.text = 'front - %r' % obj.front
    label3.text = 'position - %r' % obj.position
    
    label0.draw()
    label1.draw()  
    label2.draw()  
    label3.draw()  

    hello_world.draw()

In [17]:
#app.get_logging_widget()

In [ ]:
app.run()